In [1]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 65kB/s 
     |████████████████████████████████| 204kB 37.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=edc89a8f53022da82822d1b1bfb5ff85b57d72c5c4f9530d935fa927d268c34f
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

# Question 1

Load it into spark and use divide-and-conquer to find the first (adj, noun) pair in which the noun is 'unification'. Print the corresponding adjective.  The skeleton code is provided below.  One solution is to use filter() to find all pairs where the noun is 'unification', and then report the first one.  This is inefficient.  The better idea is to find, in parallel, the first such pair in each partition (if one exists), and then find the first partition that returns such a pair.

```python

numPartitions = 10

lines = sc.textFile(path_to_file, numPartitions)
pairs = lines.map(lambda l: tuple(l.split())).filter(lambda p: len(p)==2)
pairs.cache()

# FILL IN YOUR CODE HERE
```

In [10]:
numPartitions = 10

def find_word(iterator):
    for adj, noun in iterator:
        if noun == "unification":
            yield (adj, noun)
            break

path_to_file = "/content/drive/My Drive/courses/HKUST/MSBD5003/data/adj_noun_pairs.txt"
lines = sc.textFile(path_to_file, numPartitions)
pairs = lines.map(lambda l: tuple(l.split())).filter(lambda p: len(p)==2)
pairs.cache()


words = pairs.mapPartitions(find_word).collect()
print(words[0][0])

several


## Answer

```python
numPartitions = 10

def find_word(iterator):
    for adj, noun in iterator:
        if noun == "unification":
            yield (adj, noun)
            break

path_to_file = "/content/drive/My Drive/courses/HKUST/MSBD5003/data/adj_noun_pairs.txt"
lines = sc.textFile(path_to_file, numPartitions)
pairs = lines.map(lambda l: tuple(l.split())).filter(lambda p: len(p)==2)
pairs.cache()


words = pairs.mapPartitions(find_word).collect()
print(words[0][0])
```

# Question 2

Design a parallel divide-and-conquer algorithm for the following problem: Given two strings of equal length, compare them lexicographically. Output '<', '=', or '>', depending on the comparison result. The skeleton code is provided below.  Your code should run on all partitions of the rdd in parallel.

```python
x = 'abcccbcbcacaccacaabb'
y = 'abcccbcccacaccacaabb'

numPartitions = 4
rdd = sc.parallelize(zip(x,y), numPartitions)

# FILL IN YOUR CODE HERE
```

In [17]:
ord("A")

65

In [23]:
x = 'abcccbcbcacaccacaabb'
y = 'abcccbcccacaccacaabb'

def get_sum_by_partitions(iterator):
    sum_1 = 0
    sum_2 = 0
    for a, b in iterator:
        sum_1 += ord(a)
        sum_2 += ord(b)
    yield (sum_1, sum_2)
 
numPartitions = 4
rdd = sc.parallelize(zip(x,y), numPartitions)
sums = rdd.mapPartitions(get_sum_by_partitions).collect()

sum_1 = 0
sum_2 = 0

for s_1, s_2 in sums:
    sum_1 += s_1
    sum_2 += s_2

if sum_1 < sum_2:
    print("<")

elif sum_1 == sum_2:
    print("=")

else:
    print(">")


>


## Answer

```python
x = 'abcccbcbcacaccacaabb'
y = 'abcccbcccacaccacaabb'

def get_sum_by_partitions(iterator):
    sum_1 = 0
    sum_2 = 0
    for a, b in iterator:
        sum_1 += ord(a)
        sum_2 += ord(b)
    yield (sum_1, sum_2)
 
numPartitions = 4
rdd = sc.parallelize(zip(x,y), numPartitions)
sums = rdd.mapPartitions(get_sum_by_partitions).collect()

sum_1 = 0
sum_2 = 0

for s_1, s_2 in sums:
    sum_1 += s_1
    sum_2 += s_2

if sum_1 < sum_2:
    print("<")

elif sum_1 == sum_2:
    print("=")

else:
    print(">")

```